In [36]:
import numpy as np
import pandas as pd
import os
import openpyxl
import tensorflow as tf
from openpyxl import load_workbook
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from pandas import DataFrame
from sklearn.preprocessing import MinMaxScaler
from pandas import concat
from tensorflow.keras.layers import LSTM,Dense,GRU,Dropout,SimpleRNN
from sklearn.metrics import mean_squared_error

<!-- # 주은이랑 주식에도 있느넫, 

012345 가 있으면 
data 2 dlaus 
01
12
23
45
이렇게 된다. 

2차원이다. 
23 45 67
11 33 44
23 44 67
data = 2 가 되면 

23 56 77 11 33 44
11 33 44 34 55 67

순차적으로 데이터를 넣기 위해서 그런것이다 .

유입량만 예측하고 싶은 것이다 .

23 56 77
11 33. 4 
34 55 67

에서 23 11. 34 밑에 있는 거을 예측하고 싶으면, 

창을 만들때 , 

 -->

In [37]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [38]:
n_hours = 5
n_features = 43
train_num_24 = 2789
train_num_25 = 2892
train_num_19 = 1934

In [39]:
from os.path import join
df = pd.read_excel('rain_data.xlsx')
df.head()

,홍수사상번호,연,월,일,시간,유입량,데이터집단 1,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 38,Unnamed: 39,Unnamed: 40,데이터 집단 6,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47
0,1,2006,7,10,8,189.100000,6.4,7,7,7,...,8,2.54,122.660,6.4,7,7,8,8,2.54,122.610
1,1,2006,7,10,9,216.951962,6.3,7,8,7,...,10,2.53,122.648,7.3,7,8,10,10,2.53,122.600
2,1,2006,7,10,10,251.424419,6.4,7,9,7,...,11,2.53,122.636,8.2,7,9,10,11,2.53,122.590
3,1,2006,7,10,11,302.812199,7.3,7,10,7,...,14,2.53,122.620,11.3,9,10,15,14,2.53,122.585
4,1,2006,7,10,12,384.783406,8.2,7,12,8,...,16,2.53,122.604,14.4,12,12,18,16,2.53,122.575


주희 여기에서 실수 했다 . 

In [40]:
# df_metrix 만드는 과정
df_matrix = df.to_numpy()
df_matrix = df_matrix[0:train_num_25, 5:]
# 5 가 의미하는 것은 홍수 번호, 연, 월 일 시간 을 의미하는것 -2 는 뒤에 뭐가 있ㅇ는거 
# 맨 앞은 뭔가 잇ㄷ아고 한다. 
#train_num_25 까지 잘랐다고 한다.  26을 뺀거라고 이해하면 ㄷㅚㄴ다. 
# df_matrix = df_matrix.astype('float32')
# df_matrix

In [41]:
# scaler = MinMaxScaler(feature_range=(0, 1))
# df_matrix = scaler.fit_transform(df_matrix)



In [42]:
print(df_matrix)
# df_matrix, n_hours =  
# 
lag = series_to_supervised(df_matrix, n_hours,1)
lag = lag.values

tmp = lag.shape[1]-42
# lag.shape 가 전체 칼럼 수이다 .그 중 t-1,2,3, 이 올것이다. 그 t 에서 유입량만 들어오게 ㄷㅚㄴ다. 

lag = lag[:,0:tmp]

print(lag.shape)

[[189.1         6.4         7.        ...   8.          2.54
  122.61     ]
 [216.9519618   6.3         7.        ...  10.          2.53
  122.6      ]
 [251.4244187   6.4         7.        ...  11.          2.53
  122.59     ]
 ...
 [481.1030831   3.3854      1.        ...   1.          2.96
  137.165    ]
 [470.4581089   3.2841      1.        ...   1.          2.94
  137.18     ]
 [        nan  14.2576     32.        ...   0.          1.93
  120.515    ]]
(2886, 216)


In [43]:
n_train_hours = train_num_19
train = lag[:n_train_hours, :]
test = lag[n_train_hours:, :]

라벨링 할때 쓰는 것이라고 한다. 
test 는 19~25 번 를 넣은 것이다. 

In [56]:
n_obs = n_hours * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]
print(train_X.shape, len(train_X), train_y.shape)
print("---------------------------------------------------------------------------")
print("train_X")
print(len(train_X))
print(train_X)

print("---------------------------------------------------------------------------")
print("train_y")
print(len(train_y))
print(train_y)
print("---------------------------------------------------------------------------")
print("test_X")
print(len(test_X))
print(test_X)
print("---------------------------------------------------------------------------")
print("test_y")
print(len(test_y))
# print(test_y)

(1934, 215) 1934 (1934,)
---------------------------------------------------------------------------
train_X
1934
[[189.1          6.4          7.         ...  16.           2.53
  122.575     ]
 [216.9519618    6.3          7.         ...  17.           2.53
  122.565     ]
 [251.4244187    6.4          7.         ...  21.           2.53
  122.555     ]
 ...
 [929.6042269   86.4227      79.         ...   6.           1.95
  130.07      ]
 [917.2956892   86.3195      79.         ...   3.           1.94
  130.035     ]
 [ 63.57928546  60.7854      50.         ...   0.           1.94
  130.005     ]]
---------------------------------------------------------------------------
train_y
1934
[ 8.2    11.3    14.4    ... 59.8524 58.5964 57.6765]
---------------------------------------------------------------------------
test_X
952
[[ 65.96683605  60.7854      50.         ...   0.           1.93
  129.975     ]
 [ 69.46720861  59.8524      50.         ...   0.           1.92
  129.94      ]
 [

394536 은 1934 * 204 인 상황이다. 즉, train_X 의 크기를 의미하는 것이다 .

In [49]:
# reshape input to be 3D [samples, timesteps, features]
print("before reshape")
print("train_X.shape[0], n_hours, n_features")
print(train_X.shape[0], n_hours, n_features)

print("\nafter reshape")
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
print("train_X.shape[0], n_hours, n_features")
print(train_X.shape[0], n_hours, n_features)
print("\ntrain_X = train_X.reshaped")
print("\ntrain_X.shape = train_X.reshape")
print(train_X)
print(train_X)

print("\n")
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))
# print("train_X.shape, train_y.shape, test_X.shape, test_y.shape")
# print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)


before reshape
train_X.shape[0], n_hours, n_features
1934 5 43

after reshape
train_X.shape[0], n_hours, n_features
1934 5 43

train_X = train_X.reshaped

train_X.shape = train_X.reshape
[[[189.1          6.4          7.         ...   8.           2.54
   122.61      ]
  [216.9519618    6.3          7.         ...  10.           2.53
   122.6       ]
  [251.4244187    6.4          7.         ...  11.           2.53
   122.59      ]
  [302.812199     7.3          7.         ...  14.           2.53
   122.585     ]
  [384.7834064    8.2          7.         ...  16.           2.53
   122.575     ]]

 [[216.9519618    6.3          7.         ...  10.           2.53
   122.6       ]
  [251.4244187    6.4          7.         ...  11.           2.53
   122.59      ]
  [302.812199     7.3          7.         ...  14.           2.53
   122.585     ]
  [384.7834064    8.2          7.         ...  16.           2.53
   122.575     ]
  [512.5012279   11.3          7.         ...  17.           2.5

In [22]:
#testing
model = tf.keras.Sequential()
model.add(GRU(units = 50,
              # dropout =0.2,
              return_sequences = True,
              input_shape = (train_X.shape[1],train_X.shape[2]),
              activation = 'tanh'))
model.add(SimpleRNN(50))
model.add(Dense(1))

In [53]:
print("train_X.shape")
print(train_X.shape)
print("train_y")
print(train_y.shape)
print("test_X")
print(test_X.shape)
print("test_y")
print(test_y.shape)

train_X.shape
(1934, 5, 43)
train_y
(1934,)
test_X
(952, 5, 43)
test_y
(952,)


In [23]:
adam = tf.keras.optimizers.Adam(lr =0.01)

model.compile(loss='mse', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=90, batch_size=100, validation_data=(test_X, test_y), verbose=2, shuffle=False)

Epoch 1/90
20/20 - 0s - loss: 9666.7715 - val_loss: 4039.2771
Epoch 2/90
20/20 - 0s - loss: 9146.2920 - val_loss: 3805.7651
Epoch 3/90
20/20 - 0s - loss: 8847.0342 - val_loss: 3677.2634
Epoch 4/90
20/20 - 0s - loss: 8656.0518 - val_loss: 3582.9285
Epoch 5/90
20/20 - 0s - loss: 8505.0996 - val_loss: 3502.1399
Epoch 6/90
20/20 - 0s - loss: 8368.7666 - val_loss: 3426.7290
Epoch 7/90
20/20 - 0s - loss: 8240.2148 - val_loss: 3354.8523
Epoch 8/90
20/20 - 0s - loss: 8117.5889 - val_loss: 3283.8843
Epoch 9/90
20/20 - 0s - loss: 7999.1440 - val_loss: 3215.7217
Epoch 10/90
20/20 - 0s - loss: 7884.1616 - val_loss: 3148.7432
Epoch 11/90
20/20 - 0s - loss: 7772.3228 - val_loss: 3084.0291
Epoch 12/90
20/20 - 0s - loss: 7662.4888 - val_loss: 3021.8762
Epoch 13/90
20/20 - 0s - loss: 7553.8872 - val_loss: 2962.3972
Epoch 14/90
20/20 - 0s - loss: 7448.0679 - val_loss: 2904.6426
Epoch 15/90
20/20 - 0s - loss: 7345.4722 - val_loss: 2849.6392
Epoch 16/90
20/20 - 0s - loss: 7245.5952 - val_loss: 2793.6350
E

In [24]:
pred = model.predict(test_X)


In [25]:
RMSE = mean_squared_error(test_y,pred)**0.5
print(RMSE)

28.518814746944052


In [14]:
plt.plot(history.history['loss'],label = 'train')
plt.plot(history.history['val_loss'], label = 'test')

plt.show()


plt.plot(pred,label='prediction')
plt.plot(test_y,label='real')
plt.show()


NameError: name 'history' is not defined